In [1]:
# Lots of headers, though some may be unused rn
import cv2
import math
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
from keras.preprocessing import image
import numpy as np
from keras.utils import np_utils
from skimage.transform import resize
from sklearn.model_selection import train_test_split
from glob import glob
from tqdm import tqdm

import keras
from keras.models import Sequential
from keras.applications.vgg16 import VGG16
from keras.layers import Dense, InputLayer, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, GlobalMaxPooling2D

from moviepy.editor import VideoFileClip

Using TensorFlow backend.


Imageio: 'ffmpeg-linux64-v3.3.1' was not found on your computer; downloading it now.
Try 1. Download from https://github.com/imageio/imageio-binaries/raw/master/ffmpeg/ffmpeg-linux64-v3.3.1 (43.8 MB)
Downloading: 8192/45929032 bytes (0.0%)1458176/45929032 bytes (3.2%)5259264/45929032 bytes (11.5%)9101312/45929032 bytes (19.8%)12943360/45929032 bytes (28.2%)16801792/45929032 bytes (36.6%)20553728/45929032 bytes (44.8%)23830528/45929032 bytes (51.9%)27607040/45929032 bytes (60.1%)31440896/45929032 bytes (68.5%)35241984/45929032 bytes (76.7%)39157760/45929032 bytes (85.3%)42926080/45929032 bytes (93.5%)

Part 1: Partition the video into frames and save within a temp folder.

In [2]:
# Uses google colab for this, you may have to use your own.
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
cd drive/'My Drive'/'DLP Program'

/content/drive/My Drive/DLP Program


In [0]:
videoFile = ""
input(videoFile)

In [0]:
# Breaks the video by frames for training set
cap = cv2.VideoCapture(videoFile)
x=1; count = 0
while(cap.isOpened()):
    # Current frame number
    frameId = cap.get(1)
    ret, frame = cap.read()
    if (ret != True):
        break
    # Saves every 10th frame in the video, can be changed 
    if (frameId % 10 == 0): 
        filename ='temp/' + videoFile.split(' ')[0] +"_frame%d.jpg" % count;
        count+=1
        cv2.imwrite(filename, frame)
cap.release()

Part 2: Prepare the model we chose, this is what ours will look like for now

In [0]:
# Uses the base model of pre-trained VGG16 model
base_model = VGG16(weights='imagenet', include_top=False)

# Defining the architecture 
model = Sequential()
model.add(Dense(1000, activation='relu', input_shape=(25088,)))
model.add(Dropout(0.5))
model.add(Dense(500, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))

In [0]:
# compiling the model
model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])

Part 3: Begin analyzing the frames!

In [0]:
# creating two lists to store predicted and actual tags
predict = []

images = glob("temp/*.jpg")
prediction_images = []
for i in range(len(images)):
    img = image.load_img(images[i], target_size=(224,224,3))
    img = image.img_to_array(img)
    img = img/255
    prediction_images.append(img)
prediction_images = np.array(prediction_images)
prediction_images = base_model.predict(prediction_images)
prediction_images = prediction_images.reshape(prediction_images.shape[0], 7*7*512)
prediction = model.predict(prediction_images)

In [0]:
# Get timestamps
clip = VideoFileClip(videoFile)
timestamps = np.arange(0.0, clip.duration, clip.duration/(len(prediction)))

# Get prediction values here
prediction = prediction.transpose()
pockets = prediction[2]
erratic = prediction[1]
drumming = prediction[0]

In [0]:
# Plot here!
plt.title('Prediction results')
plt.xlabel('Time (seconds)')
plt.ylabel('Guess (%)')

plt.plot(timestamps, pockets)
plt.plot(timestamps, erratic)
plt.plot(timestamps, drumming)

plt.savefig(videoFile.split('.')[0] + '_Predictions.png')

In [0]:
# Creates the json here!
import json
data1 = list(zip(timestamps, pockets))
data2 = list(zip(timestamps, erratic))
data3 = list(zip(timestamps, drumming))

data1 = [list([float(i) for i in ele]) for ele in data1] 
data2 = [list([float(i) for i in ele]) for ele in data2] 
data3 = [list([float(i) for i in ele]) for ele in data3]

print(data1)
dicttemp = {'pockets':data1,'erratic':data2,'drumming':data3}
with open(videoFile.split('.')[0] + '_data.json', 'w') as f:
    json.dump(dicttemp, f)